In [ ]:
import json
import requests
import pymysql
from datetime import datetime
from datetime import timedelta
import pandas as pd
import sqlalchemy
import numpy as np
import unicodedata
import re   
import sqlalchemy
#weather information
#from pyowm import OWM
#from pyowm.utils import config
#from pyowm.utils import timestamps

def lambda_handler(event, context):
    
    # TODO implement
    
    schema="gans"
    host="wbs-proj.cqtnnk1ywmvs.us-east-1.rds.amazonaws.com"
    user="admin"
    password="April2022*"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    get_flights(con)
    get_weather_info(con)
    
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

    #def lambda_handler(event, context):
    # connect to database
    # cnx = pymysql.connect(
     #    user='admin',
     #    password='April2022*',
     #    host='wbs-proj.cqtnnk1ywmvs.us-east-1.rds.amazonaws.com',
     #    database='test_1')
          
   #  cursor = cnx.cursor()
  
    # insert data to table
   #   data_to_insert = (
   #      "INSERT INTO test_table "
   #       "(FirstName, City) "
   #      "VALUES ('Lambda', 'Cloud')")
  
   #  cursor.execute(data_to_insert)
  
    # commit changes & close connection
    # cnx.commit()
    # cursor.close()
    # cnx.close()
    
    
   # use test1;
   # select * from test_table;
def get_flights(con):
    #API key from the AeroDataBoxAPI
    API_key = "d97cf8dff6msh485edce0ed50d58p16d0e3jsnd30885c1218a"

    # Airports icoa
    airport_icoa = "EDDB"

    #company wants to know which flights will arrive the next day -Date Format.
    #Add 1 day.
    tomorrows_date = (datetime.now() + timedelta(days=1))
    tomorrows_date_from=tomorrows_date.strftime("%Y-%m-%dT%H:%M")
    #add 1day + 11 hours
    future_date_and_time = tomorrows_date  + timedelta(hours=11)
    tomorrows_date_to = future_date_and_time.strftime("%Y-%m-%dT%H:%M")
    
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{tomorrows_date_from}/{tomorrows_date_to}"
    querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}
    
    headers = {
    	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
    	"X-RapidAPI-Key": API_key
    }
    
    response = requests.request("GET", url, headers=headers, params=querystring)
            
    arrivals_berlin = response.json()['arrivals']
    
    def get_flight_info(flight_json):
        # terminal
        try: terminal = flight_json['arrival']['terminal']
        except: terminal = None
        # aircraft
        try: aircraft = flight_json['aircraft']['model']
        except: aircraft = None
    
        return {
            'dep_airport':flight_json['departure']['airport']['name'],
            'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
            'terminal':terminal,
            'status':flight_json['status'],
            'aircraft':aircraft,
            'icao_code':airport_icoa
        }
    
    
    # [get_flight_info(flight) for flight in arrivals_berlin]
    arrivals_berlin = pd.DataFrame([get_flight_info(flight) for flight in arrivals_berlin])

    (
    arrivals_berlin
        .replace({np.nan},'unknown')
        .assign(sched_arr_loc_time = lambda x: pd.to_datetime(x['sched_arr_loc_time']))
        .to_sql('arrivals_berlin', if_exists='append', con=con, index=False))
        
def get_weather_info(con):
    OWM_key = 'e9be608790c21e8bdbc240b2b4a3e9bc'
    city = 'Berlin'
    country = 'DE'
    
    response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')
    forecast_api = response.json()['list']
    # look for the fields that could ve relevant: 
    # better field descriptions https://www.weatherbit.io/api/weather-forecast-5-day

    weather_info = []
    
    # datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
    for forecast_3h in forecast_api: 
        weather_hour = {}
        # datetime utc
        weather_hour['datetime'] = forecast_3h['dt_txt']
        # temperature 
        weather_hour['temperature'] = forecast_3h['main']['temp']
        # wind
        weather_hour['wind'] = forecast_3h['wind']['speed']
        # probability precipitation 
        try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
        except: weather_hour['prob_perc'] = 0
        # rain
        try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
        except: weather_hour['rain_qty'] = 0
        # wind 
        try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
        except: weather_hour['snow'] = 0
        weather_hour['municipality_iso_country'] = city + ',' + country
        weather_info.append(weather_hour)
        
    weather_data = pd.DataFrame(weather_info)
    weather_data.assign(datetime = lambda x: pd.to_datetime(x['datetime'])).to_sql('weather', if_exists='append', con=con, index=False)
    
